In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
from google.colab import files

# Load and preprocess data
try:
    commodity_data = pd.read_csv('/content/aggregated_daily_data_Azadpur_Onion_commodity2023-2018.csv')
    weather_data = pd.read_csv('/content/aggregated_weather_data.csv')
except Exception as e:
    print(f"Error loading data: {e}")
    raise

commodity_data['Arrival_Date'] = pd.to_datetime(commodity_data['Arrival_Date'])
price_data = commodity_data[['Arrival_Date', 'Modal_Price']].rename(columns={'Arrival_Date': 'ds', 'Modal_Price': 'y'})

weather_data['Date'] = pd.to_datetime(weather_data['Date'])
weather_data = weather_data[['Date', 'T', 'P0', 'P', 'U', 'Ff', 'VV', 'Td', 'DD', 'WW']]
merged_data = pd.merge(price_data, weather_data, left_on='ds', right_on='Date', how='left').drop('Date', axis=1)

# Fill missing values
merged_data.fillna(merged_data.mean(numeric_only=True), inplace=True)
merged_data['DD'] = merged_data['DD'].fillna(merged_data['DD'].mode()[0])
merged_data['WW'] = merged_data['WW'].fillna(merged_data['WW'].mode()[0])

# Encode categorical features
merged_data['DD_encoded'] = pd.factorize(merged_data['DD'])[0]
merged_data['WW_encoded'] = pd.factorize(merged_data['WW'])[0]
merged_data.drop(['DD', 'WW'], axis=1, inplace=True)

# Check for NaN values
if merged_data.isna().any().any():
    print("Warning: Some NaN values were found in the merged data.")

# Split data
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42, shuffle=False)

# Train Prophet model
model = Prophet()
model.add_regressor('T')
model.add_regressor('P0')
model.add_regressor('U')
model.add_regressor('Ff')
model.add_regressor('VV')
model.add_regressor('Td')
model.add_regressor('DD_encoded')
model.add_regressor('WW_encoded')
model.fit(train_data)

# Make future predictions
forecast_horizon = 243
future = model.make_future_dataframe(periods=forecast_horizon)

# Ensure future DataFrame includes all columns
future = future.merge(merged_data[['ds', 'T', 'P0', 'U', 'Ff', 'VV', 'Td', 'DD_encoded', 'WW_encoded']], on='ds', how='left')

# Fill future data with means where missing
for col in ['T', 'P0', 'U', 'Ff', 'VV', 'Td', 'DD_encoded', 'WW_encoded']:
    future[col] = future[col].fillna(merged_data[col].mean())

# Check for NaN values before prediction
if future.isna().any().any():
    print("Warning: Some NaN values were found in the future DataFrame.")

forecast = model.predict(future)
predicted_prices = forecast[['ds', 'yhat']]

# Filter and save results
start_date = pd.to_datetime('2024-01-01')
end_date = pd.to_datetime('2024-09-08')
filtered_predictions = predicted_prices[(predicted_prices['ds'] >= start_date) & (predicted_prices['ds'] <= end_date)]

output_file_path = '/content/Predicted.csv'
filtered_predictions.to_csv(output_file_path, index=False)
files.download(output_file_path)

# Evaluate performance on the test set
test_predictions = model.predict(test_data)
mae = mean_absolute_error(test_data['y'], test_predictions['yhat'])
print(f"Mean Absolute Error: {mae}")

# Optional: Check other metrics if needed
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

mse = mean_squared_error(test_data['y'], test_predictions['yhat'])
mape = mean_absolute_percentage_error(test_data['y'], test_predictions['yhat'])

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Percentage Error: {mape}")


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmppsi6yey1/i2x_3vuv.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmppsi6yey1/q1armdxv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=49436', 'data', 'file=/tmp/tmppsi6yey1/i2x_3vuv.json', 'init=/tmp/tmppsi6yey1/q1armdxv.json', 'output', 'file=/tmp/tmppsi6yey1/prophet_modelr2xrrpgb/prophet_model-20240908160300.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:03:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:03:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mean Absolute Error: 652.7714911174952
Mean Squared Error: 655025.2278348213
Mean Absolute Percentage Error: 0.6186136157184946
